In [1]:
import datajoint as dj
# Display all config settings
# dj.config
# http://ucsd-demo-helium.datajoint.io/login

In [3]:
dj.config

{   'connection.charset': '',
    'connection.init_function': None,
    'database.hose': 'ucsd-demo-db.datajoint.io',
    'database.host': 'ucsd-demo-db.datajoint.io',
    'database.password': 'qqqqqqq',
    'database.port': 3306,
    'database.reconnect': False,
    'database.user': 'alex',
    'display.limit': 7,
    'display.show_tuple_count': True,
    'display.width': 14,
    'history': [   'No config file found, using default settings.',
                   'Updated from config file: dj_local_conf.json'],
    'loglevel': 'INFO',
    'safemode': True}

In [4]:
schema = dj.schema('alex_test')

Connecting alex@ucsd-demo-db.datajoint.io:3306


In [14]:
# Must DROP to change the table.
# AFTER you MUST RERUN `class Mouse(..)`
#Mouse.drop()
#Person.drop()
#Surgery.drop()

@schema
class Mouse(dj.Manual):
    definition = """
    mouse : char(10)                # Name for lab mouse, max 8 chars
    -------
    rating_out_of_ten=0 : int        # Useless number, must be int
    iacuc_barcode  : bigint        # Unique barcode identifier
    date_of_birth  : date          # The mouse's date of birth
    sex            : enum('M','F') # Either 'M' for male, 'F' for female
    genotype       : varchar(1000) # String indicating the genotype
    
    unique index (iacuc_barcode)   # Adds constraint, barcode must be unique accross mice
    """
@schema
class Person(dj.Lookup):
    definition = """
    username : varchar(16)
    ----------
    full_name : varchar(30)
    """
    # For lookup tables you must provide contents
    contents = [
        ('david','David Kleinfeld'),
        ('conrad', 'Conrad Foo'),
        ('cajal', 'Santiago Ramon y Cajal'),
        ('alex', 'Alexander Newberry')
    ]

@schema
class Surgery(dj.Manual):
    definition = """
    -> Mouse
    surgery_number : tinyint unsigned  # unsigned int frmo 0-56
    ----------------
    -> Person
    surgery_date : date
    surgery_description = "" : varchar(1000)
    """

In [18]:
Mouse.insert1(dict(
    mouse=1, 
    iacuc_barcode='11734', 
    date_of_birth='2018-10-25', 
    sex='F', 
    genotype='C56',
    ), skip_duplicates=True)
Mouse.insert1(dict(
    mouse=1, 
    iacuc_barcode='11755', 
    date_of_birth='2018-10-27', 
    sex='M', 
    genotype='C56'
    ), skip_duplicates=True)
# IF you change the definition of Mouse() (add or take away items)
#  you must run Mouse.drop() to remove the old table.
Mouse()

"mouse Name for lab mouse, max 8 chars","rating_out_of_ten Useless number, must be int",iacuc_barcode Unique barcode identifier,date_of_birth The mouse's date of birth,"sex Either 'M' for male, 'F' for female",genotype String indicating the genotype
1,0,11734,2018-10-25,F,C56
2,0,11755,2018-10-27,M,C56


In [28]:
Surgery.insert1(dict(
    mouse=1, 
    surgery_number=1, 
    username='alex', 
    surgery_date='2018-11-11',
    surgery_description='Nothing to say.'
    ), skip_duplicates=True)
Surgery.insert1(dict(
    mouse=1, 
    surgery_number=2, 
    username='alex', 
    surgery_date='2018-11-12',
    surgery_description='Surgery went horribly, horribly wrong. That poor mouse.'
    ), skip_duplicates=True)
Surgery.insert1(dict(
    mouse=1, 
    surgery_number=3, 
    username='david', 
    surgery_date='2018-11-14',
    surgery_description='Surgery was chillin'
    ), skip_duplicates=True)

Surgery()

"mouse Name for lab mouse, max 8 chars",surgery_number unsigned int frmo 0-56,username,surgery_date,surgery_description
1,1,alex,2018-11-11,Nothing to say.
1,2,alex,2018-11-12,"Surgery went horribly, horribly wrong. That poor mouse."
1,3,david,2018-11-14,Surgery was chillin


In [29]:
Surgery() & dict(username='alex')

"mouse Name for lab mouse, max 8 chars",surgery_number unsigned int frmo 0-56,username,surgery_date,surgery_description
1,1,alex,2018-11-11,Nothing to say.
1,2,alex,2018-11-12,"Surgery went horribly, horribly wrong. That poor mouse."


In [31]:
Surgery() & dict(username='alex', surgery_number='2')

"mouse Name for lab mouse, max 8 chars",surgery_number unsigned int frmo 0-56,username,surgery_date,surgery_description
1,2,alex,2018-11-12,"Surgery went horribly, horribly wrong. That poor mouse."


In [33]:
male_mice = Mouse() & dict(sex='F')

In [3]:
# Visualize the current nodes
# Requires matplotlib
dj.ERD(schema)

NameError: name 'schema' is not defined